# Second assignment of Research Track II

## Jupyter notebook of the node user_interface

With this implementation you will be able control the behavior of the robot within the Gazebo environment. In addition, you will have much more functionalities to control the simulation thanks to the presence of widgets and the Matplotlib library. 

In particular the new functionalities, implemented thanks to these tools, are now:
- the ability to start/stop the robot, that is regulated with two buttons;
- the ability of changing the velocities, both linear and angular, using two sliders;
- the ability of directly teleoperate the movements of the robot in each direction (plus the stop), with 5 buttons.

Moreover, you have some information, concerning the simulation, that are graphically displayed, including:
- a line plot that displays cmd_vel vs actual velocity (for both linear and angular velocities);
- a bar plot that allows to see the number of reached and canceled target positions during the execution of the simulation;
- a hist plot that shows the time required to reach each target position;
- an xy graph that shows the robot's position and orientation during the execution of the simulation.

## How to run the code

After cloning the repository in a ROS workspace, you'll need two different terminals.

In the first terminals you need to type the following instructions:
- in your ROS workspace you need to build the package that you have just cloned using the command 

   `catkin_make`
   
   
- then, you need to go inside the package that you have cloned and starts the simulation by typing

   `roslaunch rt2_assignment1 notebook.launch`

In the second terminal you need to start the Jupyter notebook by typing

`jupyter notebook --allow-root --ip 0.0.0.0`


Then open the file **user_interface.ipynb** and run all the cells (the ones that you need) to be able to directly interact with the Gazebo simulation environment.


Advice: if you have not already done so, I recommend that you activate the nbextention *execute time*, to make sure that a cell has actually been run. 

## Headers and initialization of the node

This notebook, as it happens for the user_interface node, communicates with the other three nodes that are the:
- go_to_point; 
- position_service;
- state_machine.

Here are listed the headers that are needed for our purpose:

In [1]:
import rospy
import time
import math
from rt2_assignment1.srv import Command
import actionlib
import actionlib.msg
import rt2_assignment1.msg
from rt2_assignment1.msg import TargetActionResult
from std_msgs.msg import Int32

import matplotlib.pyplot as plt
import numpy as np
import jupyros as jr
import ipywidgets as widgets
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry
from tf import transformations
from matplotlib import animation, rc
from matplotlib.animation import FuncAnimation
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox, FloatSlider
from IPython.display import display
%matplotlib widget

Now we initialize the node and implement the action client in order to start and stop the behavior of the robot, without waiting the end of the task:

In [2]:
# initialization of the node
rospy.init_node('user_interface')
ui_client = rospy.ServiceProxy('/user_interface', Command)
    
# action client to cancel the goal 
act_c = actionlib.SimpleActionClient("go_to_point", rt2_assignment1.msg.TargetAction)
act_c.wait_for_server()

True

## Global variables and functions

Then the global variable used within the notebook are all initialized here and, moreover, here we can find the callback to the /odom topic, in a way that we can know the real position of the robot in any time, and as a consequence also the real velocity, the callback to the /cmd_vel topic, to know the velocity given to the robot, and the target callback, that advertise when the robot reaches its goal position.

In [3]:
twist_msg = Twist()
time_init = time.time()

# time variables
current_time = 0
total_time = []

# counter variables
start_count = 0
stop_count = 0

# actual linear and angular velocities
lin_cmd_data = []
ang_cmd_data = []

# real linear and angular velocities
lin_odom_data = []
ang_odom_data = []

# position variables
x_pos_data = []
y_pos_data = []

# orientation variables
yaw = 0
x_orient_data = []
y_orient_data = []
arrow_mod = 0.8

start_time = 0
stop_time = 0
time_needed = []

# reached and cancelled targets
reached = 0
canceled = 0 
num = [0,0]

def odom_clbk(msg):
    global time_init, current_time, total_time, x_pos_data, y_pos_data, yaw, lin_odom_data, ang_odom_data
    # time
    current_time = time.time() - time_init
    total_time.append(current_time)
    # real position of the robot
    x_pos_data.append(msg.pose.pose.position.x)
    y_pos_data.append(msg.pose.pose.position.y)

    # real yaw of the robot
    quaternion = (
        msg.pose.pose.orientation.x,
        msg.pose.pose.orientation.y,
        msg.pose.pose.orientation.z,
        msg.pose.pose.orientation.w)
    euler = transformations.euler_from_quaternion(quaternion)
    yaw = euler[2]
    
    # real linear velocity
    lin_odom_data.append(msg.twist.twist.linear.x)
    # real angular velocity
    ang_odom_data.append(msg.twist.twist.angular.z)
    
    # if the lists of elements collected are too big then delete the first element from each list
    if len(total_time)>300:
        total_time.pop(0)
    if len(x_pos_data) > 300 and len(y_pos_data) > 300:
        x_pos_data.pop(0)
        y_pos_data.pop(0)
    if len(lin_odom_data) > 300 and len(ang_odom_data) > 300:
        lin_odom_data.pop(0)
        ang_odom_data.pop(0)
           
    
def cmd_clbk(msg):
    global lin_cmd_data, ang_cmd_data
    # linear and angular velocity given to the robot
    lin_cmd_data.append(msg.linear.x)
    ang_cmd_data.append(msg.angular.z)

    if len(lin_cmd_data) > 300 and len(ang_cmd_data) > 300:
        lin_cmd_data.pop(0)
        ang_cmd_data.pop(0)
        
        
def target_clbk(msg):
    global reached, canceled, num, start_time, stop_time, time_needed
    if(msg.data == 1):
        reached += 1
        stop_time = time.time()
        time_needed.append(stop_time - start_time)
        start_time = time.time()
    else:
        canceled += 1
    num = [reached, canceled]
        
        
# publisher to cmd_vel topic and subscriber to the odom topic nad cmd_vel topic
pub = rospy.Publisher('/cmd_vel', Twist, queue_size = 1)
odom_sub = rospy.Subscriber('/odom', Odometry, odom_clbk)
cmd_sub = rospy.Subscriber('/cmd_vel', Twist, cmd_clbk)
target_sub = rospy.Subscriber('robot_target', Int32, target_clbk)

### Starting/stopping the robot using buttons

With this implementation, you will have two different buttons, that allow to interact directly with the simulation, making the robot starts and stops while the it is moving to reach its random position target within the simulation environment.
Each of the button has a callback function associated, so anytime one of the button is pressed by the user, the corresponding callback function will be executed.

In [4]:
start_button = widgets.Button(description='START', layout=widgets.Layout(widht='auto',grid_area='b1'),
                style=widgets.ButtonStyle(button_color='limegreen')) 
stop_button = widgets.Button(description='STOP', layout=widgets.Layout(widht='auto',grid_area='b2'),
                style=widgets.ButtonStyle(button_color='firebrick')) 

In [5]:
output = widgets.Output()

def start_on(b):
    global start_time
    # start the robot behavior
    ui_client('start')
    start_time = time.time()    
    with output:
        print("The robot has been started")


def stop_on(b):
    # stop the robot behavior
    act_c.cancel_all_goals()
    ui_client('stop')    
    with output:
        print("The robot has been stopped")
    
    
start_button.on_click(start_on) # on_click method for the start button
stop_button.on_click(stop_on) # on_click method for the stop button
display(output) # display the message

Output()

In [6]:
# arrange the buttons neatly
widgets.GridBox(children=[start_button,stop_button],
                layout=widgets.Layout(width='60%',grid_template_rows='auto auto',
                grid_template_columns='30% 20% 30%',
                grid_template_areas=''' "b1 . b2" '''))

GridBox(children=(Button(description='START', layout=Layout(grid_area='b1'), style=ButtonStyle(button_color='l…

### Changing the linear and the angular velocity using sliders

With this implementation you will have two different sliders, that allow to change the linear and the angular velocity while the simulation is running. 
In particular, the user can select velocities between -1.0 and 1.5 m/s and rad/s. The default velocity without using the sliders is for both 0.5 m/s and rad/s.

In [7]:
lin_vel_init = 0.5
lin_vel_slider = widgets.FloatSlider(description='Linear velocity', value=lin_vel_init, min=-1.0, max=1.5)
lin_vel_slider.style.handle_color = 'blue'

ang_vel_init = 0.5
ang_vel_slider = widgets.FloatSlider(description='Angular velocity', value=ang_vel_init, min=-1.0, max=1.5)
ang_vel_slider.style.handle_color = 'blue'

In [8]:
def lin_vel_change(change): 
    global twist_msg
    twist_msg.linear.x = change['new']
    pub.publish(twist_msg)
    
def ang_vel_change(change):
    global twist_msg
    twist_msg.angular.z = change['new']
    pub.publish(twist_msg)
    
lin_vel_slider.observe(lin_vel_change, names='value')
ang_vel_slider.observe(ang_vel_change, names='value')

In [9]:
widgets.HBox([widgets.VBox([lin_vel_slider, ang_vel_slider])])

### Teleoperate the robot using buttons

With this implementation you will have 5 different buttons. 
Four of them are use to directly teleoperate the robot within the simulation and are the following ones:
- ↑ to go forward
- ↓ to go backward
- ← to turn left
- → to turn right
The other button is:
- STOP to stop the robot

As in the previous case, at each button is associated a callback, that is executed each time that the button get pressed.

In [14]:
go_forward_button = widgets.Button(description='\u2191',layout=Layout(width='auto', grid_area='b1'),
style=ButtonStyle(button_color='lavender'))
go_backward_button = widgets.Button(description='\u2193',layout=Layout(width='auto', grid_area='b2'),
style=ButtonStyle(button_color='lavender'))
turn_left_button = widgets.Button(description='\u2190',layout=Layout(width='auto', grid_area='b3'),
style=ButtonStyle(button_color='lavender'))
turn_right_button = widgets.Button(description='\u2192',layout=Layout(width='auto', grid_area='b4'),
style=ButtonStyle(button_color='lavender'))
stop_button = widgets.Button(description='STOP',layout=Layout(width='auto', align="center", grid_area='b5'),
style=ButtonStyle(button_color='firebrick'))

In [15]:
def go_forward(click):
    global twist_msg
    twist_msg.linear.x = 1.0
    twist_msg.angular.z = 0.0
    pub.publish(twist_msg)
    
def go_backward(click):
    global twist_msg
    twist_msg.linear.x = - 1.0
    twist_msg.angular.z = 0.0
    pub.publish(twist_msg)
    
def turn_left(click):
    global twist_msg
    twist_msg.linear.x = 0.0
    twist_msg.angular.z = -1.0
    pub.publish(twist_msg)
    
def turn_right(click):
    global twist_msg
    twist_msg.linear.x = 0.0
    twist_msg.angular.z = 1.0
    pub.publish(twist_msg)
    
def stop_behavior(behavior):
    global twist_msg
    act_c.cancel_all_goals()
    ui_client("stop")
    twist_msg.linear.x = 0.0
    twist_msg.angular.z = 0.0
    pub.publish(twist_msg)
    
go_forward_button.on_click(go_forward)
go_backward_button.on_click(go_backward)
turn_left_button.on_click(turn_left)
turn_right_button.on_click(turn_right)
stop_button.on_click(stop_behavior)

In [16]:
widgets.GridBox(children=[go_forward_button, go_backward_button, turn_left_button, turn_right_button ,stop_button],
                layout=widgets.Layout(width='75%',grid_template_rows='auto auto auto',
                grid_template_columns='15% 15% 15% 20% 35%',
                grid_template_areas='''
                ". b1 . . ."
                "b3 b2 b4 . ."
                " . . . . b5"
                ''')
               )

GridBox(children=(Button(description='↑', layout=Layout(grid_area='b1', width='auto'), style=ButtonStyle(butto…

## Displaying graphical information

### Line plot: cmd_vel vs actual velocity

The following plots show the comparison between the requested velocity and the actual velocity, respectively for the linear and the angular velocities. In addition, the method FuncAnimation is used: it makes an animation by repeatedly calling a function, that in this case are the animate_lin_vel and the animate_ang_vel, that updates the animation every time it is called.

In [7]:
fig_lin_vel, ax_lin_vel = plt.subplots()
fig_ang_vel, ax_ang_vel = plt.subplots()

lin_vel_cmd, = ax_lin_vel.plot([], [], color='magenta', label = 'lin_vel_cmd')
lin_vel_odom, = ax_lin_vel.plot([], [], color='navy', label = 'lin_vel_odom')

ang_vel_cmd, = ax_ang_vel.plot([], [], color='mediumturquoise', label = 'ang_vel_cmd')
ang_vel_odom, = ax_ang_vel.plot([], [], color='olivedrab', label = 'ang_vel_odom')

ax_lin_vel.set_title('Linear velocity: cmd_vel vs odom')
ax_lin_vel.set(xlabel = 'Time [s]', ylabel = 'Linear velocity [m/s]', xlim = (current_time - 10, current_time + 10), ylim = (-1.2, 1.7))

ax_ang_vel.set_title('Angular velocity: cmd_vel vs odom')
ax_ang_vel.set(xlabel = 'Time [s]', ylabel = 'Angular velocity [rad/s]', xlim = (current_time - 10, current_time + 10), ylim = (-1.2, 1.7))

ax_lin_vel.legend()
ax_ang_vel.legend()

t = time.time()
t_vel = []

def init_lin_vel():
    lin_vel_cmd.set_data([], [])
    lin_vel_odom.set_data([], [])
    return (lin_vel_cmd, lin_vel_odom)

def animate_lin_vel(i):
    global lin_cmd_data, lin_odom_data, t_vel, t
    t_vel.append(time.time() -t)
    lin_vel_cmd.set_data(t_vel, lin_cmd_data)
    lin_vel_odom.set_data(t_vel, lin_odom_data)
    return (lin_vel_cmd, lin_vel_odom)

def init_ang_vel():
    ang_vel_cmd.set_data([], [])
    ang_vel_odom.set_data([], [])
    return (ang_vel_cmd, ang_vel_odom)

def animate_ang_vel(i):
    global ang_cmd_data, ang_odom_data, t_vel, t
    t_vel.append(time.time() -t)
    ang_vel_cmd.set_data(t_vel, ang_cmd_data)
    ang_vel_odom.set_data(t_vel, ang_odom_data)
    return (ang_vel_cmd, ang_vel_odom)

anim_lin_vel = animation.FuncAnimation(fig_lin_vel, animate_lin_vel, init_func=init_lin_vel, frames=100, interval=20, blit=True)
anim_ang_vel = animation.FuncAnimation(fig_ang_vel, animate_ang_vel, init_func=init_ang_vel, frames=100, interval=20, blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Bar plot: reached and canceled targets

With this plot you're going to visualize how many target positions have been reached by the robot within the simulation and how many targets have been canceled by the user.
In addition, the method FuncAnimation is used: it makes an animation by repeatedly calling a function, that in this case is the animate_barplot, that updates the animation every time it is called.

In [11]:
fig_barplot, ax_barplot = plt.subplots()

ax_barplot.set_title('Number of reached and canceled target positions')
ax_barplot.set(ylabel = 'Number of targets')

targets = np.array(['Reached','Canceled'])
barplot = ax_barplot.bar(targets, [0,0], width = 0.5, color = 'teal')

def init_barplot():
    ax_barplot.bar(targets, [0,0], width = 0.5, color = 'teal')
    return (ax_barplot)
             
def animate_barplot(i):
    global targets, num
    ax_barplot.bar(targets, num, width = 0.5, color = 'teal')
    return (ax_barplot)


anim_bar = animation.FuncAnimation(fig_barplot, animate_barplot, init_func = init_barplot, frames=100, interval=10, blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Hist plot: required time to reach the targets

With this plot you're going to visualize the time required, in seconds, to reach each target within the simulation.
In addition, the method FuncAnimation is used: it makes an animation by repeatedly calling a function, that in this case is the animate_histplot, that updates the animation every time it is called.

In [12]:
fig_histplot, ax_histplot = plt.subplots()
ax_histplot.set_title('Time needed to reach the target')
ax_histplot.set(xlabel = 'Time [s]')

def init_histplot():
    ax_histplot.hist([], bins = 50, color = 'orange')
    return (ax_histplot)
    
def animate_histplot(i):
    global time_needed
    ax_histplot.hist(time_needed, bins = 50, color = 'orange')
    return (ax_histplot)


anim_hist = animation.FuncAnimation(fig_histplot, animate_histplot, init_func = init_histplot, frames=100, interval=10, blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### xy graph: robot's position and orientation

With this graph you are able to visualize the trajectory that the robot is following in order to reach the target position in the simulation environment. So, you are going to see both the real-time position in the space and also the orientation of the robot. The orientation is highlighted by an arrow.  

In [13]:
fig_pos, ax_pos = plt.subplots()

pos_plt, = ax_pos.plot([], [], 'orchid')

ax_pos.set_title('Position and orientation of the robot')
ax_pos.set(xlabel = 'x', ylabel = 'y', xlim = (-7,7) , ylim = (-7,7))

def init_pos():
    pos_plt.set_data([], [])
    return (pos_plt, arrow)

def animate_pos(i):
    global x_pos_data, y_pos_data, yaw, x_orient_data, y_orient_data
    pos_plt.set_data(x_pos_data, y_pos_data)
    if len(x_pos_data) > 0:
        x = x_pos_data[len(x_pos_data) - 1]
        y = y_pos_data[len(y_pos_data) - 1]
        
        arrow.set_position((x,y))
        x_orient_data = np.cos(yaw)
        y_orient_data = np.sin(yaw)
        arrow.xy = (x + arrow_mod*x_orient_data, y + arrow_mod*y_orient_data)
        
    return pos_plt, arrow

arrow = ax_pos.annotate('', xy = (arrow_mod*np.cos(0), arrow_mod*np.sin(0)), xytext = (0,0), arrowprops = dict(facecolor = 'darkviolet', shrink = 0.05))

anim_pos = animation.FuncAnimation(fig_pos, animate_pos, init_func = init_pos, frames = 100, interval = 20, blit = True)
fig_pos.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …